# HOMEWORK 6: TEXT CLASSIFICATION
In this homework, you will create models to classify texts from TRUE call-center. There are two classification tasks:
1. Action Classification: Identify which action the customer would like to take (e.g. enquire, report, cancle)
2. Object Classification: Identify which object the customer is referring to (e.g. payment, truemoney, internet, roaming)

We will focus only on the Object Classification task for this homework.

In this homework, you are asked compare different text classification models in terms of accuracy and inference time.

You will need to build 3 different models.

1. A model based on tf-idf
2. A model based on MUSE
3. A model based on wangchanBERTa

**You will be ask to submit 3 different files (.pdf from .ipynb) that does the 3 different models. Finally, answer the accuracy and runtime numbers in MCV.**

This homework is quite free form, and your answer may vary. We hope that the processing during the course of this assignment will make you think more about the design choices in text classification.

In [1]:
# !wget --no-check-certificate https://www.dropbox.com/s/37u83g55p19kvrl/clean-phone-data-for-students.csv
# !pip install pythainlp

## Import Libs

In [2]:
%matplotlib inline
import pandas
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from torch.utils.data import Dataset
from IPython.display import display
from collections import defaultdict
from sklearn.metrics import accuracy_score

#My import 
np.random.seed(42)
from sklearn.model_selection import train_test_split


## Loading data
First, we load the data from disk into a Dataframe.

A Dataframe is essentially a table, or 2D-array/Matrix with a name for each column.

In [3]:
data_df = pd.read_csv('clean-phone-data-for-students.csv')

Let's preview the data.

In [4]:
# Show the top 5 rows
display(data_df.head())
# Summarize the data
data_df.describe()

,Sentence Utterance,Action,Object
0,<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counte...,enquire,payment
1,internet ยังความเร็วอยุ่เท่าไหร ครับ,enquire,package
2,ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้...,report,suspend
3,พี่ค่ะยังใช้ internet ไม่ได้เลยค่ะ เป็นเครื่อ...,enquire,internet
4,ฮาโหล คะ พอดีว่าเมื่อวานเปิดซิมทรูมูฟ แต่มันโ...,report,phone_issues


,Sentence Utterance,Action,Object
count,16175,16175,16175
unique,13389,10,33
top,บริการอื่นๆ,enquire,service
freq,97,10377,2525


## Data cleaning

We call the DataFrame.describe() again.
Notice that there are 33 unique labels/classes for object and 10 unique labels for action that the model will try to predict.
But there are unwanted duplications e.g. Idd,idd,lotalty_card,Lotalty_card

Also note that, there are 13389 unqiue sentence utterances from 16175 utterances. You have to clean that too!

## #TODO 0.1:
You will have to remove unwanted label duplications as well as duplications in text inputs.
Also, you will have to trim out unwanted whitespaces from the text inputs.
This shouldn't be too hard, as you have already seen it in the demo.



In [5]:
display(data_df.describe())
display(data_df.Object.unique())
display(data_df.Action.unique())

,Sentence Utterance,Action,Object
count,16175,16175,16175
unique,13389,10,33
top,บริการอื่นๆ,enquire,service
freq,97,10377,2525


array(['payment', 'package', 'suspend', 'internet', 'phone_issues',
       'service', 'nonTrueMove', 'balance', 'detail', 'bill', 'credit',
       'promotion', 'mobile_setting', 'iservice', 'roaming', 'truemoney',
       'information', 'lost_stolen', 'balance_minutes', 'idd',
       'TrueMoney', 'garbage', 'Payment', 'IDD', 'ringtone', 'Idd',
       'rate', 'loyalty_card', 'contact', 'officer', 'Balance', 'Service',
       'Loyalty_card'], dtype=object)

array(['enquire', 'report', 'cancel', 'Enquire', 'buy', 'activate',
       'request', 'Report', 'garbage', 'change'], dtype=object)

In [6]:
# TODO.1: Data cleaning

# select only the columns that we need
data_df = data_df[['Sentence Utterance', 'Object']]
data_df.columns = ['input', 'raw_label']

# remove duplicate labels
data_df['clean_label'] = data_df['raw_label'].str.lower().copy()
data_df.drop('raw_label', axis=1, inplace=True)

# remove duplicate input rows
data_df = data_df.drop_duplicates("input", keep="first")
display(data_df.describe())

,input,clean_label
count,13389,13389
unique,13389,26
top,สอบถามโปรโมชั่นปัจจุบันที่ใช้อยู่ค่ะ,service
freq,1,2111


In [7]:
# map label to number and convert to numpy "data"
data = data_df.to_numpy()
unique_label = data_df.clean_label.unique()

label_2_num_map = dict(zip(unique_label, range(len(unique_label))))
num_2_label_map = dict(zip(range(len(unique_label)), unique_label))

# convert label to number
data[:,1] = np.vectorize(label_2_num_map.get)(data[:,1])

In [8]:
# Input string cleaning
def strip_str(string):
    return string.strip()
     
# Trim of extra begining and trailing whitespace in the string
data[:,0] = np.vectorize(strip_str)(data[:,0])

### Keep in mind class is imbalance type shi

Split data into train, valdation, and test sets (normally the ratio will be 80:10:10 , respectively). We recommend to use train_test_spilt from scikit-learn to split the data into train, validation, test set.

In addition, it should split the data that distribution of the labels in train, validation, test set are similar. There is **stratify** option to handle this issue.

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

Make sure the same data splitting is used for all models.

In [9]:
data

array([['<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counter Services เค้าเช็ต 3276.25 บาท เมื่อวานที่ผมเช็คที่ศูนย์บอกมียอด 3057.79 บาท',
        0],
       ['internet ยังความเร็วอยุ่เท่าไหร ครับ', 1],
       ['ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้ ค่ะ', 2],
       ...,
       ['ยอดเงินเหลือเท่าไหร่ค่ะ', 7],
       ['ยอดเงินในระบบ', 7],
       ['สอบถามโปรโมชั่นปัจจุบันที่ใช้อยู่ค่ะ', 1]], dtype=object)

In [10]:
X = data[:,0]
y = data[:,1]

#handle class 24 with only 4 samples
X_class_24 = X[y == 24]
y_class_24 = y[y == 24]

random_idx = np.random.choice(len(X_class_24), 2, replace=False)
X_class_24_val, X_class_24_test = X_class_24[random_idx[0]], X_class_24[random_idx[1]]
X_class_24_train = np.delete(X_class_24, random_idx)
y_class_24_val, y_class_24_test = y_class_24[random_idx[0]], y_class_24[random_idx[1]]
y_class_24_train = np.delete(y_class_24, random_idx)


# remove class 24 from the data to add it later
X = X[y != 24]
y = y[y != 24]

# train val test split 80:10:10
X_train, X_tmp, y_train, y_tmp = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_tmp, y_tmp, test_size=0.5, stratify=y_tmp, random_state=42)

# add class 24 to the train, val, test set
X_train = np.append(X_train, X_class_24_train)
y_train = np.append(y_train, y_class_24_train)
X_val = np.append(X_val, X_class_24_val)
y_val = np.append(y_val, y_class_24_val)
X_test = np.append(X_test, X_class_24_test)
y_test = np.append(y_test, y_class_24_test)

# use numpy save to save the data
np.save('./split_data/X_train.npy', X_train)
np.save('./split_data/y_train.npy', y_train)
np.save('./split_data/X_val.npy', X_val)
np.save('./split_data/y_val.npy', y_val)
np.save('./split_data/X_test.npy', X_test)
np.save('./split_data/y_test.npy', y_test)


#check label distribution
display(np.unique(y_train, return_counts=True))
display(np.unique(y_val, return_counts=True))
display(np.unique(y_test, return_counts=True))


(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25], dtype=object),
 array([ 513, 1438,  584, 1432,  465, 1689,  197, 1185,  262,  432,  138,
         916,  225,   17,  197,  198,  237,  185,   40,  165,   39,   63,
          29,   54,    2,    8]))

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25], dtype=object),
 array([ 64, 179,  73, 179,  58, 211,  24, 148,  32,  54,  18, 114,  28,
          3,  24,  25,  30,  23,   5,  21,   5,   8,   4,   7,   1,   1]))

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25], dtype=object),
 array([ 64, 180,  73, 179,  58, 211,  25, 149,  33,  54,  17, 115,  28,
          2,  25,  25,  30,  23,   5,  20,   5,   8,   3,   6,   1,   1]))

# Model 1 TF-IDF

Build a model to train a tf-idf text classifier. Use a simple logistic regression model for the classifier.

For this part, you may find this [tutorial](https://scikit-learn.org/stable/auto_examples/text/plot_document_classification_20newsgroups.html#sphx-glr-auto-examples-text-plot-document-classification-20newsgroups-py) helpful.

Below are some design choices you need to consider to accomplish this task. Be sure to answer them when you submit your model.

What tokenizer will you use? Why?

**Ans:** deepcut . Since it seems to perform the best when on the BEST val,test set benchmark

Will you ignore some stop words (a, an, the, to, etc. for English) in your tf-idf? Is it important?
PythaiNLP provides a list of stopwords if you want to use (https://pythainlp.org/docs/2.0/api/corpus.html#pythainlp.corpus.common.thai_stopwords)

**Ans:**

The dictionary of TF-IDF is usually based on the training data. How many words in the test set are OOVs?

**Ans:**

In [11]:
X_train

array(['เดือนละ 150 บาทเล่นได้ทั้งวันหรือเปล่า',
       'ครับ มีการชำระค่าบริการมาแล้ว ดูสัญญาณให้หน่อย',
       'จ่ายค่าทรูมูฟค่ะ สัญญาณใช้ได้วันนี้เลยหรือเปล่าค่ะ', ...,
       'คร้าบ คือว่าเบอร์มันโดนระงับชั่วคราวอ่ะครับ แล้วตะกี้เพิ่งไปจ่ายมาอ่ะครับ',
       'ได้รับข้อความแจ้งเตือนการใช้งาน',
       'พอดีพี่จะสอบถาม พี่เห็นคัสโตเมอร์แคร์โทรหาพี่หลายครั้งแล้ว มีอะไรหรือเปล่าค่า'],
      dtype=object)

In [ ]:
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus.common import thai_stopwords

# Tokenize the input string
word_tokenize(X_train[0], engine="deepcut")


# Model 2 MUSE

Build a simple logistic regression model using features from the MUSE model.

Which MUSE model will you use? Why?

**Ans:**

MUSE is typically used with tensorflow. However, there are some pytorch conversions made by some people.

https://huggingface.co/sentence-transformers/use-cmlm-multilingual
https://huggingface.co/dayyass/universal-sentence-encoder-multilingual-large-3-pytorch

# Model 3 WangchanBERTa

We ask you to train a WangchanBERTa-based model.

We recommend you use the thaixtransformers fork (which we used in the PoS homework).
https://github.com/PyThaiNLP/thaixtransformers

The structure of the code will be very similar to the PoS homework. You will also find the huggingface [tutorial](https://huggingface.co/docs/transformers/en/tasks/sequence_classification) useful. Or you can also add a softmax layer by yourself just like in the previous homework.

Which WangchanBERTa model will you use? Why? (Don't forget to clean your text accordingly).

**Ans:**


After you

# Comparison

After you have completed the 3 models, compare the accuracy, ease of implementation, and inference speed (from cleaning, tokenization, till model compute) between the three models in mycourseville.